In [6]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [7]:
# read in shapefile file
'''
read in shapefile -- filepaths below
Kevan: r'C:\\Users\kevan\\OneDrive\\Desktop\\Data+\\DevCasesSHP\\Development_Cases.shp'
Leah: '/Users/leahwallihan/Durham_school_planning/geospatial files/Res_Development'
'''
all_cases = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/Res_Development') 

In [8]:
# types = A_TYPE codes we want to keep
types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP'] 
# filter that only keeps cases of specified types
filter_cases_type = all_cases[all_cases['A_TYPE'].isin(types)]
# status = A_STATUS codes we want to keep
status = all_cases['A_STATUS'].unique()
status = status[~np.isin(status, ['WITH', 'VOID','DEN','DISAP','EXP'])]
# filter that only keeps cases of specified status
filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

In [9]:
keywords = ['home', 'family', 'residen', 'mixed', 'mized', 'duplex', 'apartment', ' housing', 'condo', 'dwelling', 'tenant', 'affordable', 'units', 'townhouse']
pattern = '|'.join(keywords)
filtered1 = filter_cases_status[filter_cases_status['A_DESCRIPT'].str.contains(pattern, case=False, na=False)]

#use not pattern to avoid these words
keywords_avoid = ['expand','storage']
pattern_avoid = '|'.join(keywords_avoid)
filtered2 = filtered1[~filtered1['A_DESCRIPT'].str.contains(pattern_avoid, case=False, na=False)]

In [10]:
#filter for all status dates after 2020
filtered2['A_STATUS_D'] = pd.to_datetime(filtered2['A_STATUS_D'])
filtered3 = filtered2[filtered2['A_STATUS_D'].dt.year>=2020]

/opt/anaconda3/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [11]:
#define extract_units function number 1

# def normalize_for_regex(term):
#     return re.sub(r'[-\s]+', r'\\s*-?\\s*', term)

# def extract_units1(description):
#     # Remove square footage
#     description = re.sub(
#         r'(\d+|\d{1,3}(,\d{3})*)(\s+[A-Za-z-]+){0,2}?\s*(SF|square feet|sq\.?\s*ft\.?|sqft)',
#         '', description, flags=re.IGNORECASE
#     )

#     # Housing normalization, THIS MEANS THAT IF HOUSES->HOME, S-F -> SINGLE FAMILY, ETC.
#     term_map = {
#         "home": "home", "homes": "home", "house": "home", "houses": "home",
#         "duplex": "duplex", "duplexes": "duplex",
#         "condo": "condo", "condominium": "condo", "condominiums": "condo", "condos": "condo", 
#         "apartment": "apartment", "apartments": "apartment",
#         "townhome": "townhouse", "townhomes": "townhouse",
#         "townhouse": "townhouse", "townhouses": "townhouse",
#         "town home": "townhouse", "town homes": "townhouse",
#         "town house": "townhouse", "town houses": "townhouse",
#         "multifamily": "multifamily", "multi-family": "multifamily", "multi - family": "multifamily", "multi family": "multifamily",
#         "MF": "multifamily", "mutifamily": "multifamily",
#         "single family": "single family", "single-family": "single family", 
#         "single - family": "single family", "s-f": "single family", "s - f": "single family", "s f": "single family"
#     }

#     # Optional leading and trailing terms
#     modifiers = ["attached", "detached"]
#     suffixes = ["units", "lots", "homes", "houses"]

#     # Build regex patterns
#     housing_pattern = "|".join([normalize_for_regex(term) for term in term_map])
#     modifier_pattern = "|".join(modifiers)
#     suffix_pattern = "|".join(suffixes)

#     match_pattern = rf'''
#         \b
#         (?P<qty>\d{{1,4}})
#         \s+
#         (?:(?P<mod>{modifier_pattern})\s+)?
#         (?:[A-Za-z-]+\s+){{0,2}}?
#         (?P<type>{housing_pattern})
#         (?:\s+(?P<suffix>{suffix_pattern}))?
#         \b
#     '''

#     matches = re.finditer(match_pattern, description, flags=re.IGNORECASE | re.VERBOSE)
    
#     result = []
#     for match in matches:
#         qty = match.group("qty")
#         raw_type = match.group("type")
#         raw_mod = match.group("mod")
#         raw_suffix = match.group("suffix")

#         # Normalize type
#         norm_key = re.sub(r'[-\s]+', ' ', raw_type.lower()).strip()
#         normalized_type = term_map.get(norm_key, norm_key)

#         # Build output tuple
#         result.append((
#             int(qty),
#             raw_mod.lower() if raw_mod else None,
#             normalized_type,
#             raw_suffix.lower() if raw_suffix else None
#         ))

#     return result

In [108]:
def normalize_for_regex(term):
    return re.sub(r'[-\s]+', r'\\s*-?\\s*', term)

def extract_units(description):
    # Remove square footage references
    description = re.sub(
        r'(\d+|\d{1,3}(,\d{3})*)(\s+[A-Za-z-]+){0,2}?\s*(SF|square feet|sq\.?\s*ft\.?|sqft)',
        '', description, flags=re.IGNORECASE
    )

    # Map variations to standardized types
    term_map = {
        "home": "home", "homes": "home", "house": "home", "houses": "home",
        "duplex": "duplex", "duplexes": "duplex",
        "condo": "condo", "condominium": "condo", "condominiums": "condo", "condos": "condo", 
        "apartment": "apartment", "apartments": "apartment",
        "townhome": "townhouse", "townhomes": "townhouse",
        "townhouse": "townhouse", "townhouses": "townhouse",
        "town home": "townhouse", "town homes": "townhouse",
        "town house": "townhouse", "town houses": "townhouse",
        "multifamily": "multifamily", "multi-family": "multifamily", 
        "multi - family": "multifamily", "multi family": "multifamily",
        "mutifamily": "multifamily", "MF": "multifamily",
        "single family": "single family", "single-family": "single family", 
        "single - family": "single family", "s-f": "single family", "s - f": "single family", "s f": "single family"
    }

    modifiers = ["attached", "detached"]
    suffixes = ["units", "lots", "homes", "houses"]

    housing_pattern = "|".join([normalize_for_regex(term) for term in term_map])
    modifier_pattern = "|".join(modifiers)
    suffix_pattern = "|".join(suffixes)

    # Extended match pattern to support both "qty before type" and "type before qty"
    match_pattern = rf'''
    (?:
        # Qty before type
        (?P<qty>\(?\d{{1,4}}\)?)
        (?:\s*[-+&/]?\s*)?
        (?:({modifier_pattern})\s*){{0,2}}?
        (?:\w+\s*){{0,4}}?
        (?P<type>{housing_pattern})
        (?:\s+({modifier_pattern}))?
        (?:\s+(?P<suffix>{suffix_pattern}))?

    |
        # Type before qty
        (?P<type2>{housing_pattern})
        (?:\s+({modifier_pattern}))?
        (?:\s*[-+&/]?\s*)?
        (?:\w+\s*){{0,4}}?
        (?P<qty2>\(?\d{{1,4}}\)?)
        (?:\s+(?P<suffix2>{suffix_pattern}))?

    |
        # Type with quantity in parentheses
        (?P<type3>{housing_pattern})
        (?:\s+\w+){{0,4}}?
        \(\s*(?P<qty3>\d{{1,4}})\s+(?P<suffix3>{suffix_pattern})\s*\)
    )
'''

    matches = re.finditer(match_pattern, description, flags=re.IGNORECASE | re.VERBOSE)
    
    result = []
    for match in matches:
        qty = match.group("qty") or match.group("qty2")
        raw_type = match.group("type") or match.group("type2")
        raw_mod = match.group(2)  # first modifier (position varies)
        raw_suffix = match.group("suffix") or match.group("suffix2")

        if not qty or not raw_type:
            continue  # skip malformed matches

        # Normalize type
        norm_key = re.sub(r'[-\s]+', ' ', raw_type.lower()).strip()
        normalized_type = term_map.get(norm_key, norm_key)

        result.append((
            int(qty.strip("()")),
            raw_mod.lower() if raw_mod else None,
            normalized_type,
            raw_suffix.lower() if raw_suffix else None
        ))

    return result


In [109]:
filtered3['match_results'] = filtered3['A_DESCRIPT'].apply(extract_units)
# filtered3['match_results'] = filtered3['A_DESCRIPT'].apply(extract_units2)
filtered3 = filtered3.to_crs('EPSG:4326')

In [110]:
# create a column for each relevant housing type
housing_types = ['sf_detached', 'sf_attached', 'duplex/triplex', 'multifamily', 'condo']
for h_type in housing_types: 
    filtered3[h_type] = 0

housing_type_dict = {
        'townhouse': 'sf_attached',
        'home': 'sf_detached', 'single family': 'sf_detached', 
        'duplex': 'duplex/triplex', 
        'apartment': 'multifamily', 'multifamily': 'multifamily', 
        'condo': 'condo'}
      
    
# function to fill in housing type columns
def fill_types(match_results):

    row_data = {h_type: 0 for h_type in housing_types}
    for group in match_results:
        
        quantity = int(group[0])
        mod = group[1] if len(group) > 1 else None
        housing = group[2] if len(group) > 2 else None
        
        if housing == 'single family' and mod == 'attached':
            row_data['sf_attached'] += quantity
        elif housing in housing_type_dict:
            row_data[housing_type_dict[housing]] += quantity
        else:
            pass

    return pd.Series(row_data)
            

In [111]:
filtered3[housing_types] = filtered3['match_results'].apply(fill_types)

In [112]:
'''
read in highschool boundaries shapefile -- file paths:
Kevan: r'C:\\Users\\kevan\\OneDrive\\Desktop\\Data+\\regions_2025_26\\regions_2025_26.shp'
Leah: '/Users/leahwallihan/Durham_school_planning/geospatial files/HS_regions'
'''
high_school_boundaries = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/HS_regions') 
high_school_boundaries = high_school_boundaries.to_crs('EPSG:4326')

In [113]:
filtered3_with_school = filtered3.copy()

for i,geometry in enumerate(high_school_boundaries['geometry']):
    in_geometry = geometry.contains(filtered3['geometry'])
    high_school_name = high_school_boundaries.loc[i, 'region']

    filtered3_with_school.loc[in_geometry, 'region'] = high_school_name
    

In [114]:
'''
read in planning unit shapefile -- file paths: 
Leah: /Users/leahwallihan/Durham_school_planning/geospatial files/pu_shape.geojson
'''

pu_shape = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/pu_shape.geojson')
pu_shape = pu_shape.to_crs('EPSG:4326')

In [115]:
# assign planning units to developments
filtered_with_shapes = filtered3_with_school.copy()

for i, geometry in enumerate(pu_shape['geometry']):
    in_geometry = geometry.contains(filtered3_with_school['geometry'])
    pu = pu_shape.loc[i, 'OBJECTID']

    filtered_with_shapes.loc[in_geometry, 'pu'] = pu

In [116]:
# filtered_with_shapes.to_file('resdev_cases.geojson', driver='GeoJSON')

In [117]:
'''
read in csv file for DPS development data -- file paths: 
Leah: /Users/leahwallihan/Durham_school_planning/CSV files/DPS_residential_developments.csv
'''

DPS_resdev = pd.read_csv('/Users/leahwallihan/Durham_school_planning/CSV files/DPS_residential_developments.csv')
DPS_resdev[['A_DESCRIPTION', 'PLNS_NUMBER_OF_LOTS', 'PLNS_NUMBER_OF_UNITS', 'PLNS_APARTMENTS', 'PLNS_TOWNHOMES', 'PLNS_CONDOMINIUMS']].head(20)

,A_DESCRIPTION,PLNS_NUMBER_OF_LOTS,PLNS_NUMBER_OF_UNITS,PLNS_APARTMENTS,PLNS_TOWNHOMES,PLNS_CONDOMINIUMS
0,36 proposed townhomes.,NaN,225.0,T,NaN,NaN
1,"Changes to sub-phasing, lot depths and street l, main amenities site, impervious area, open spac...",NaN,NaN,NaN,NaN,NaN
2,"various revisions made with this submittal including changing of alley section, adding sidewalk ...",NaN,NaN,NaN,NaN,NaN
3,"Changes to building layout, parking, and sidewalks. Addition of one 4-car garage building. No ch...",52.0,NaN,NaN,T,NaN
4,"New townhome development with 53 units and new right of way, in association with Z2300045",NaN,NaN,NaN,T,NaN
5,Building footprint update with no additional impervious.,NaN,NaN,NaN,NaN,NaN
6,Changes to amenity area.,NaN,NaN,NaN,NaN,NaN
7,160 unit affordable apartment.,NaN,NaN,NaN,NaN,NaN
8,"208 Single-family, 186 townhouses, 8 apartments residential development. 402 Total Units",NaN,90.0,T,NaN,NaN
9,"Update the Site Plan to the approved CDs (Stormwater & Utilities), additional outdoor dining are...",NaN,NaN,NaN,NaN,NaN


In [118]:
pd.set_option('display.max_colwidth', 100)
filtered_with_shapes[['A_DESCRIPT', 'sf_detached', 'sf_attached', 'duplex/triplex', 'multifamily', 'condo']].head(20)

,A_DESCRIPT,sf_detached,sf_attached,duplex/triplex,multifamily,condo
90,"50 Single - family lots, 1 stormwater pond, ad associated infrastructure",50,0,0,0,0
139,Mass grading only site plan for residential townhome project.,0,0,0,0,0
224,Mixe of uses including retail and residential. Zoned DD-C,0,0,0,0,0
242,Multifamily Apartments,0,0,0,0,0
265,"37 Townhome units, 102 Attached S-F units (called Duplex), 146 Detached S-F lots",248,37,0,0,0
290,"From Rural Density Residential (RDR, 0.5DU/acre or less) to Very Low Density Residential (VLDR, ...",0,0,0,0,0
308,33 new town home lots with garages and mail kiosk with parking,0,33,0,0,0
336,"32 single family units on small lots and 23 townhomes with associated streets, sidewalks, gradin...",32,23,0,0,0
478,"108 Single - Family lots, 2 stormwater ponds, and associated infrastructure. Pending Annexation ...",108,0,0,0,0
545,"397,000 Total SF: Apartments (264 units), Accessory Uses, Retail uses. Per D-Plan Z1500007",0,0,0,264,0
